<!--
 * @Author: Chuyang Su cs4570@columbia.edu
 * @Date: 2025-12-01 11:19:48
 * @LastEditors: Schuyn 98257102+Schuyn@users.noreply.github.com
 * @LastEditTime: 2025-12-01 11:20:08
 * @FilePath: /Unsupervised-Machine-Learning-Final-Project/Code/EDA.ipynb
 * @Description: 
 * 不要为了努力道歉
-->
## Exploratory Data Analysis

This part to fully explore the dataset and visualize the data distribution, feature relationships, and any patterns or anomalies present in the data. The goal is to gain insights that will inform subsequent modeling decisions.

###

In [3]:
'''
Author: Chuyang Su cs4570@columbia.edu
Date: 2025-12-01 11:19:48
LastEditors: Schuyn 98257102+Schuyn@users.noreply.github.com
LastEditTime: 2025-12-01 11:25:00
FilePath: /Unsupervised-Machine-Learning-Final-Project/Code/EDA.ipynb
Description: 
    EDA
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')